In [2]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

2024-07-16 13:25:41.977663: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 13:25:42.174846: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-16 13:25:42.219487: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-16 13:25:42.219505: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [20]:
df = pd.read_csv("data/data.csv")

columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'id': 12175,
 'gender': 'Female',
 'age': 54.0,
 'hypertension': 0,
 'heart_disease': 0,
 'ever_married': 'Yes',
 'work_type': 'Private',
 'Residence_type': 'Urban',
 'avg_glucose_level': 104.51,
 'bmi': 27.3,
 'smoking_status': 'smokes',
 'stroke': 1}

In [21]:
df.pop("stroke")

0       1
1       1
2       1
3       1
4       1
       ..
5105    0
5106    0
5107    0
5108    0
5109    0
Name: stroke, Length: 5110, dtype: int64

In [22]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [23]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [34]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://stroke-prediction-production-2b63.up.railway.app/v1/models/stroke-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    
    print(prediction)
    
    if prediction > 0.4:
        return "Stroke"
    else:
        return "Healthy"

In [35]:
make_predictions(inputs)

0.471199512


'Stroke'